<a href="https://colab.research.google.com/github/RajivDalal/GenAI-Practice/blob/main/ToolCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
!pip install langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [3]:
from google.colab import userdata
GEMINI_API = userdata.get('GEMINI_API')

# Tool Calling

While the name "tool calling" implies that the model is directly performing some action, the model actually only generates the arguments to a tool, and actually running the tool (or not) is up to the user.

# Defining Tool Schemas

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", api_key=GEMINI_API)

In [8]:
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


model
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]

In [13]:
model_with_tools = model.bind_tools(tools)
prompt = "What is 12 + 13"

model_with_tools.invoke(prompt)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 12.0, "b": 13.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-6e0efa1c-a9bb-422a-9a29-57515501dd10-0', tool_calls=[{'name': 'add', 'args': {'a': 12.0, 'b': 13.0}, 'id': 'dad71e76-d6b5-4fef-9f26-fcaa4e85d7e8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 35, 'output_tokens': 3, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})

In [10]:
prompt = "What is 25 + 81. Also what is 7 * 13"
model_with_tools.invoke(prompt)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 7.0, "b": 13.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-1add620a-07b9-4144-bf60-3ae06e7562b3-0', tool_calls=[{'name': 'add', 'args': {'a': 25.0, 'b': 81.0}, 'id': 'f25cf29f-5068-4813-a960-c1fbd4a83ebd', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 7.0, 'b': 13.0}, 'id': '19a426cf-1e98-4d1a-9237-41213e865e41', 'type': 'tool_call'}], usage_metadata={'input_tokens': 45, 'output_tokens': 6, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}})

In [12]:
prompt = "What is 5 * 3 + 8 * 6"
model_with_tools.invoke(prompt)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 15.0, "b": 48.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-706c5af5-ce19-4549-8c4f-fcde40e024d5-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5.0, 'b': 3.0}, 'id': 'adcd7941-eec7-420c-ae73-d5edbac8ff4c', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 8.0, 'b': 6.0}, 'id': '7ba8a408-f4fc-4fcb-ab2c-ac7eb39b5b60', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 15.0, 'b': 48.0}, 'id': '777a4825-500a-41e7-a163-502b292ceeec', 'type': 'tool_call'}], usage_metadata={'input_tokens': 39, 'output_tokens': 9, 'total_tokens': 48, 'input_token_details': {'cache_read': 0}})

# With Prompt Templates

In [17]:
from langchain_core.prompts import BasePromptTemplate
prompt_template = PromptTemplate.from_template("What is {expr}")
result = model_with_tools.invoke(prompt_template.format(expr = "12 + 13 * 2 + 7 * 4"))
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 38.0, "b": 28.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b56b35b0-4bca-4860-95cd-b2dd8a618594-0', tool_calls=[{'name': 'add', 'args': {'a': 12.0, 'b': {'a': 13.0, 'b': 2.0}}, 'id': '197dbbdc-039a-4e9c-84ac-b9985b3951c0', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 7.0, 'b': 4.0}, 'id': '365458d3-cf85-4c96-a97e-614ac6f2b4c6', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 38.0, 'b': 28.0}, 'id': 'e77010ce-11c1-4944-b99b-d3ccf80fbc20', 'type': 'tool_call'}], usage_metadata={'input_tokens': 44, 'output_tokens': 11, 'total_tokens': 55, 'input_token_details': {'cache_read': 0}})